In this notebook, I've tested three recommendation models from the RecBole framework: Bayesian Personalized Ranking (BPR), Factorization Machines (FM), and Attentional Factorization Machines (AFM), focusing on their ability to handle both implicit and explicit feedback.

**Key Findings:**
1. **BPR**: It looks like BPR doesn't really use the rating values—it just cares whether users have interacted with items or not. This means BPR might not be the best choice if we need to distinguish between different levels of user preferences, which is important for explicit feedback.
2. **FM**: This model can handle both types of feedback. However, it struggles with ranking tasks since it's primarily built for prediction. This means using FM with top-k evaluation metrics (like precision or recall) isn't straightforward and might require additional tweaks or different settings.
3. **AFM**: AFM works well with both implicit and explicit feedback thanks to its attention mechanism that highlights important interactions. But just like FM, we didn't test it with top-k metrics for explicit feedback, so we can't say for sure how well it would perform in ranking scenarios. The results also were not identical, but very similar to FM, so both methods seem to work in a similar way.

1. **BPR**:
The results from the experiments testing the BPR model with different rating configurations show that BPR performs similarly whether ratings are thresholded or all set to one. This similarity in performance suggests that for BPR, the presence of an interaction is more important than the rating's magnitude. BPR optimizes based on pairwise rankings of observed versus unobserved interactions, focusing on the presence rather than the strength of preferences. This makes BPR suitable for scenarios where explicit ratings are sparse or not available, as it effectively utilizes binary interaction data. If different levels of user satisfaction significantly impact recommendations, exploring models that can distinguish these levels or incorporating additional data might be necessary.

In [1]:
import pandas as pd
from recbole.quick_start import run_recbole

# Load and convert data to implicit feedback
data = pd.read_csv('u.data', sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])
threshold = 3.5
data['rating'] = (data['rating'] > threshold).astype(int)

# Save binary data
data.to_csv('dataset/ml100k_implicit_bpr/ml100k_implicit_bpr.inter', index=False, sep='\t', header=['user_id:token', 'item_id:token', 'rating:float', 'timestamp:float'])
# Configuration for running BPR with Implicit Feedback
config_dict_implicit = {
    'model': 'BPR',
    'dataset': 'ml100k_implicit_bpr',
    'learning_rate': 0.01,
    'epochs': 100,
    'embedding_size': 64,
    'eval_setting': 'RO_RS',
    'metrics': ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision'],
    'valid_metric': 'MRR@10',
    'topk': 10,
    'gpu_id': 0,
    'early_stop': 5,
}

# Run the model for implicit feedback
run_recbole(model='BPR', dataset='ml100k_implicit_bpr', config_dict=config_dict_implicit)

/home/stef/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/stef/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
23 May 15:41    INFO  ['/home/stef/.local/lib/python3.10/site-packages/ipykernel_launcher.py', '-f', '/home/stef/.local/share/jupyter/runtime/kernel-510548df-5f9c-4af3-8da9-9e6950dbae6c.json']
23 May 15:41    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
stat

Evaluate   :   0%|                                                          | 0/472 [00:00<?, ?it/s]:  75%|██████████████████████████████████▋           | 356/472 [00:00<00:00, 3554.44it/s]: 100%|██████████████████████████████████████████████| 472/472 [00:00<00:00, 3327.35it/s]
23 May 15:41    INFO  epoch 3 evaluating [time: 0.15s, valid_score: 0.343100]
23 May 15:41    INFO  valid result: 
recall@10 : 0.182    mrr@10 : 0.3431    ndcg@10 : 0.1981    hit@10 : 0.6893    precision@10 : 0.1393
23 May 15:41    INFO  Saving current: saved/BPR-May-23-2024_15-41-36.pth
Train     4:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  65%|███████████████████████████████▊                 | 26/40 [00:00<00:00, 257.23it/s]: 100%|█████████████████████████████████████████████████| 40/40 [00:00<00:00, 270.88it/s]
23 May 15:41    INFO  epoch 4 training [time: 0.15s, train loss: 7.1204]
Evaluate   :   0%|                                                          | 0/

Evaluate   :   0%|                                                          | 0/472 [00:00<?, ?it/s]:  69%|███████████████████████████████▊              | 327/472 [00:00<00:00, 3263.38it/s]: 100%|██████████████████████████████████████████████| 472/472 [00:00<00:00, 3129.14it/s]
23 May 15:41    INFO  epoch 12 evaluating [time: 0.16s, valid_score: 0.339800]
23 May 15:41    INFO  valid result: 
recall@10 : 0.1898    mrr@10 : 0.3398    ndcg@10 : 0.1979    hit@10 : 0.7158    precision@10 : 0.1359
Train    13:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  55%|██████████████████████████▉                      | 22/40 [00:00<00:00, 217.63it/s]: 100%|█████████████████████████████████████████████████| 40/40 [00:00<00:00, 207.64it/s]
23 May 15:41    INFO  epoch 13 training [time: 0.20s, train loss: 3.2957]
Evaluate   :   0%|                                                          | 0/472 [00:00<?, ?it/s]:  59%|███████████████████████████                

{'best_valid_score': 0.3508,
 'valid_score_bigger': True,
 'best_valid_result': OrderedDict([('recall@10', 0.1929),
              ('mrr@10', 0.3508),
              ('ndcg@10', 0.2041),
              ('hit@10', 0.7179),
              ('precision@10', 0.142)]),
 'test_result': OrderedDict([('recall@10', 0.2092),
              ('mrr@10', 0.4297),
              ('ndcg@10', 0.2439),
              ('hit@10', 0.7519),
              ('precision@10', 0.1637)])}

In [2]:
import pandas as pd
from recbole.quick_start import run_recbole

# Load the data
data = pd.read_csv('u.data', sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])

# Convert all ratings to 1
data['rating'] = 1  # Set all ratings to 1

# Save binary data
data.to_csv('dataset/ml100k_always_implicit/ml100k_always_implicit.inter', index=False, sep='\t', header=['user_id:token', 'item_id:token', 'rating:float', 'timestamp:float'])

# Configuration for running BPR
config_dict_implicit = {
    'model': 'BPR',
    'dataset': 'ml100k_always_implicit',
    'learning_rate': 0.01,
    'epochs': 100,
    'embedding_size': 64,
    'eval_setting': 'RO_RS',
    'metrics': ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision'],
    'valid_metric': 'MRR@10',
    'topk': 10,
    'gpu_id': 0,
    'early_stop': 5,
}

# Run the model for implicit feedback where all interactions are positive
run_recbole(model='BPR', dataset='ml100k_always_implicit', config_dict=config_dict_implicit)


23 May 15:43    INFO  ['/home/stef/.local/lib/python3.10/site-packages/ipykernel_launcher.py', '-f', '/home/stef/.local/share/jupyter/runtime/kernel-510548df-5f9c-4af3-8da9-9e6950dbae6c.json']
23 May 15:43    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/ml100k_always_implicit
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 100
train_batch_size = 2048
learner = adam
learning_rate = 0.01
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'RO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repea

Train     4:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  48%|███████████████████████▎                         | 19/40 [00:00<00:00, 188.39it/s]:  95%|██████████████████████████████████████████████▌  | 38/40 [00:00<00:00, 184.91it/s]: 100%|█████████████████████████████████████████████████| 40/40 [00:00<00:00, 179.36it/s]
23 May 15:43    INFO  epoch 4 training [time: 0.23s, train loss: 7.1204]
Evaluate   :   0%|                                                          | 0/472 [00:00<?, ?it/s]:  58%|██████████████████████████▌                   | 273/472 [00:00<00:00, 2722.80it/s]: 100%|██████████████████████████████████████████████| 472/472 [00:00<00:00, 2185.96it/s]
23 May 15:43    INFO  epoch 4 evaluating [time: 0.23s, valid_score: 0.343500]
23 May 15:43    INFO  valid result: 
recall@10 : 0.1842    mrr@10 : 0.3435    ndcg@10 : 0.1991    hit@10 : 0.6925    precision@10 : 0.1407
23 May 15:43    INFO  Saving current: saved/BPR-May-23-2024_15-

Train    13:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  42%|████████████████████▊                            | 17/40 [00:00<00:00, 166.45it/s]:  88%|██████████████████████████████████████████▉      | 35/40 [00:00<00:00, 173.93it/s]: 100%|█████████████████████████████████████████████████| 40/40 [00:00<00:00, 169.19it/s]
23 May 15:43    INFO  epoch 13 training [time: 0.24s, train loss: 3.2957]
Evaluate   :   0%|                                                          | 0/472 [00:00<?, ?it/s]:  35%|███████████████▉                              | 164/472 [00:00<00:00, 1636.45it/s]:  83%|██████████████████████████████████████▏       | 392/472 [00:00<00:00, 2011.17it/s]: 100%|██████████████████████████████████████████████| 472/472 [00:00<00:00, 1916.35it/s]
23 May 15:43    INFO  epoch 13 evaluating [time: 0.26s, valid_score: 0.340200]
23 May 15:43    INFO  valid result: 
recall@10 : 0.1909    mrr@10 : 0.3402    ndcg@10 : 0.1982    hit@10 : 0.71

{'best_valid_score': 0.3508,
 'valid_score_bigger': True,
 'best_valid_result': OrderedDict([('recall@10', 0.1929),
              ('mrr@10', 0.3508),
              ('ndcg@10', 0.2041),
              ('hit@10', 0.7179),
              ('precision@10', 0.142)]),
 'test_result': OrderedDict([('recall@10', 0.2092),
              ('mrr@10', 0.4297),
              ('ndcg@10', 0.2439),
              ('hit@10', 0.7519),
              ('precision@10', 0.1637)])}

2. **FM**:
The next experiment tests Factorization Machines (FMs) for both implicit and explicit feedback datasets using RecBole. A key issue encountered was the incompatibility of using top-k evaluation metrics with the FM model. This problem stems from the nature of the FM model in RecBole, which is primarily designed for prediction tasks, not ranking tasks typically assessed by top-k metrics. This mismatch led to difficulties in directly applying top-k evaluations such as Recall, Precision, and NDCG without adjustments or different model configurations that support such metrics. This issue was consistent across both implicit and explicit feedback models, indicating a broader limitation with the FM implementation in RecBole for ranking-based evaluations.

In [3]:
import pandas as pd
from recbole.quick_start import run_recbole

# Load data and normalize ratings for explicit feedback
data = pd.read_csv('u.data', sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])
max_rating = data['rating'].max()
data['rating'] = data['rating'] / max_rating

# Save the data with simple headers
data.to_csv('dataset/ml100k_explicit_fm/ml100k_explicit_fm.inter', index=False, sep='\t',
            header=['user_id:token', 'item_id:token', 'rating:float', 'timestamp:float'])

# Configuration for FM with explicit feedback
config_dict_explicit = {
    'model': 'FM',
    'dataset': 'ml100k_explicit_fm',
    'epochs': 100,
    'learning_rate': 0.01,
    'eval_setting': 'RO_RS',  # Random Ordering, Random Splitting
    'metrics': ['RMSE', 'MAE'],  # Evaluation metrics suitable for binary classification
    'valid_metric': 'RMSE',  # Main metric for validation
    'topk': 10,
    'early_stop': 5,
    'gpu_id': 0,
    'LABEL_FIELD': 'rating',  # This must match exactly with your data headers
    'load_col': {'inter': ['user_id', 'item_id', 'rating', 'timestamp']}
}

# Run the model
run_recbole(model='FM', dataset='ml100k_explicit_fm', config_dict=config_dict_explicit)

23 May 15:47    INFO  ['/home/stef/.local/lib/python3.10/site-packages/ipykernel_launcher.py', '-f', '/home/stef/.local/share/jupyter/runtime/kernel-510548df-5f9c-4af3-8da9-9e6950dbae6c.json']
23 May 15:47    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/ml100k_explicit_fm
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 100
train_batch_size = 2048
learner = adam
learning_rate = 0.01
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'RO', 'group_by': None, 'mode': {'valid': 'labeled', 'test': 'labeled'}}


23 May 15:48    INFO  Saving current: saved/FM-May-23-2024_15-47-59.pth
Train     4:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  55%|██████████████████████████▉                      | 22/40 [00:00<00:00, 211.57it/s]: 100%|█████████████████████████████████████████████████| 40/40 [00:00<00:00, 187.68it/s]
23 May 15:48    INFO  epoch 4 training [time: 0.22s, train loss: 21.7809]
Evaluate   :   0%|                                                            | 0/3 [00:00<?, ?it/s]: 100%|███████████████████████████████████████████████████| 3/3 [00:00<00:00, 266.10it/s]
23 May 15:48    INFO  epoch 4 evaluating [time: 0.02s, valid_score: 0.184300]
23 May 15:48    INFO  valid result: 
rmse : 0.1843    mae : 0.1449
Train     5:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  68%|█████████████████████████████████                | 27/40 [00:00<00:00, 267.80it/s]: 100%|████████████████████████████████████████████

+-------------+----------------+
23 May 15:48    INFO  best valid : OrderedDict([('rmse', 0.1835), ('mae', 0.1445)])
23 May 15:48    INFO  test result: OrderedDict([('rmse', 0.1847), ('mae', 0.1457)])


{'best_valid_score': 0.1835,
 'valid_score_bigger': False,
 'best_valid_result': OrderedDict([('rmse', 0.1835), ('mae', 0.1445)]),
 'test_result': OrderedDict([('rmse', 0.1847), ('mae', 0.1457)])}

In [4]:
import pandas as pd
from recbole.quick_start import run_recbole

# Load data and treat any rating as an interaction (implicit feedback)
data = pd.read_csv('u.data', sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])
data['rating'] = (data['rating'] > 0).astype(int)  # Convert ratings to 1 to denote interaction

# Save the data with simple headers
data.to_csv('dataset/ml100k_implicit_fm/ml100k_implicit_fm.inter', index=False, sep='\t',
            header=['user_id:token', 'item_id:token', 'rating:float', 'timestamp:float'])

# Configuration for FM with implicit feedback
config_dict_implicit = {
    'model': 'FM',
    'dataset': 'ml100k_implicit_fm',
    'epochs': 100,
    'learning_rate': 0.01,
    'eval_setting': 'RO_RS',  # Random Ordering, Random Splitting
    'metrics': ['RMSE', 'MAE'],  # Evaluation metrics suitable for binary classification
    'valid_metric': 'RMSE',  # Main metric for validation
    'topk': 10,
    'early_stop': 5,
    'gpu_id': 0,
    'LABEL_FIELD': 'rating',  # This must match exactly with your data headers
    'load_col': {'inter': ['user_id', 'item_id', 'rating', 'timestamp']}
}

# Run the model
run_recbole(model='FM', dataset='ml100k_implicit_fm', config_dict=config_dict_implicit)

23 May 15:48    INFO  ['/home/stef/.local/lib/python3.10/site-packages/ipykernel_launcher.py', '-f', '/home/stef/.local/share/jupyter/runtime/kernel-510548df-5f9c-4af3-8da9-9e6950dbae6c.json']
23 May 15:48    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/ml100k_implicit_fm
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 100
train_batch_size = 2048
learner = adam
learning_rate = 0.01
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'RO', 'group_by': None, 'mode': {'valid': 'labeled', 'test': 'labeled'}}


Train     4:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  65%|███████████████████████████████▊                 | 26/40 [00:00<00:00, 249.42it/s]: 100%|█████████████████████████████████████████████████| 40/40 [00:00<00:00, 226.32it/s]
23 May 15:48    INFO  epoch 4 training [time: 0.19s, train loss: 0.1110]
Evaluate   :   0%|                                                            | 0/3 [00:00<?, ?it/s]: 100%|███████████████████████████████████████████████████| 3/3 [00:00<00:00, 231.78it/s]
23 May 15:48    INFO  epoch 4 evaluating [time: 0.03s, valid_score: 0.009800]
23 May 15:48    INFO  valid result: 
rmse : 0.0098    mae : 0.0026
23 May 15:48    INFO  Saving current: saved/FM-May-23-2024_15-48-38.pth
Train     5:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  55%|██████████████████████████▉                      | 22/40 [00:00<00:00, 211.45it/s]: 100%|█████████████████████████████████████████████

Evaluate   :   0%|                                                            | 0/3 [00:00<?, ?it/s]: 100%|███████████████████████████████████████████████████| 3/3 [00:00<00:00, 253.80it/s]
23 May 15:48    INFO  epoch 14 evaluating [time: 0.02s, valid_score: 0.008600]
23 May 15:48    INFO  valid result: 
rmse : 0.0086    mae : 0.0006
23 May 15:48    INFO  Saving current: saved/FM-May-23-2024_15-48-38.pth
Train    15:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  60%|█████████████████████████████▍                   | 24/40 [00:00<00:00, 233.30it/s]: 100%|█████████████████████████████████████████████████| 40/40 [00:00<00:00, 231.69it/s]
23 May 15:48    INFO  epoch 15 training [time: 0.18s, train loss: 0.0093]
Evaluate   :   0%|                                                            | 0/3 [00:00<?, ?it/s]: 100%|███████████████████████████████████████████████████| 3/3 [00:00<00:00, 279.96it/s]
23 May 15:48    INFO  epoch 15 evaluating [time: 

Train    25:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  52%|█████████████████████████▋                       | 21/40 [00:00<00:00, 208.05it/s]: 100%|█████████████████████████████████████████████████| 40/40 [00:00<00:00, 215.73it/s]
23 May 15:48    INFO  epoch 25 training [time: 0.19s, train loss: 0.0035]
Evaluate   :   0%|                                                            | 0/3 [00:00<?, ?it/s]: 100%|███████████████████████████████████████████████████| 3/3 [00:00<00:00, 286.86it/s]
23 May 15:48    INFO  epoch 25 evaluating [time: 0.02s, valid_score: 0.008400]
23 May 15:48    INFO  valid result: 
rmse : 0.0084    mae : 0.0004
23 May 15:48    INFO  Saving current: saved/FM-May-23-2024_15-48-38.pth
Train    26:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  65%|███████████████████████████████▊                 | 26/40 [00:00<00:00, 247.65it/s]: 100%|███████████████████████████████████████████

23 May 15:48    INFO  epoch 35 evaluating [time: 0.03s, valid_score: 0.008400]
23 May 15:48    INFO  valid result: 
rmse : 0.0084    mae : 0.0004
23 May 15:48    INFO  Saving current: saved/FM-May-23-2024_15-48-38.pth
Train    36:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  55%|██████████████████████████▉                      | 22/40 [00:00<00:00, 210.33it/s]: 100%|█████████████████████████████████████████████████| 40/40 [00:00<00:00, 209.20it/s]
23 May 15:48    INFO  epoch 36 training [time: 0.20s, train loss: 0.0017]
Evaluate   :   0%|                                                            | 0/3 [00:00<?, ?it/s]: 100%|███████████████████████████████████████████████████| 3/3 [00:00<00:00, 187.73it/s]
23 May 15:48    INFO  epoch 36 evaluating [time: 0.03s, valid_score: 0.008400]
23 May 15:48    INFO  valid result: 
rmse : 0.0084    mae : 0.0004
23 May 15:48    INFO  Saving current: saved/FM-May-23-2024_15-48-38.pth
Train    37:   0%|   

...

Train    67:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  82%|████████████████████████████████████████▍        | 33/40 [00:00<00:00, 325.46it/s]: 100%|█████████████████████████████████████████████████| 40/40 [00:00<00:00, 301.91it/s]
23 May 15:48    INFO  epoch 67 training [time: 0.14s, train loss: 0.0004]
Evaluate   :   0%|                                                            | 0/3 [00:00<?, ?it/s]: 100%|███████████████████████████████████████████████████| 3/3 [00:00<00:00, 311.40it/s]
23 May 15:48    INFO  epoch 67 evaluating [time: 0.02s, valid_score: 0.008200]
23 May 15:48    INFO  valid result: 
rmse : 0.0082    mae : 0.0003
23 May 15:48    INFO  Saving current: saved/FM-May-23-2024_15-48-38.pth
Train    68:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  82%|████████████████████████████████████████▍        | 33/40 [00:00<00:00, 326.29it/s]: 100%|███████████████████████████████████████████

Evaluate   :   0%|                                                            | 0/3 [00:00<?, ?it/s]: 100%|███████████████████████████████████████████████████| 3/3 [00:00<00:00, 295.32it/s]
23 May 15:48    INFO  epoch 77 evaluating [time: 0.02s, valid_score: 0.008200]
23 May 15:48    INFO  valid result: 
rmse : 0.0082    mae : 0.0003
23 May 15:48    INFO  Saving current: saved/FM-May-23-2024_15-48-38.pth
Train    78:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  82%|████████████████████████████████████████▍        | 33/40 [00:00<00:00, 320.11it/s]: 100%|█████████████████████████████████████████████████| 40/40 [00:00<00:00, 274.03it/s]
23 May 15:48    INFO  epoch 78 training [time: 0.16s, train loss: 0.0003]
Evaluate   :   0%|                                                            | 0/3 [00:00<?, ?it/s]: 100%|███████████████████████████████████████████████████| 3/3 [00:00<00:00, 217.45it/s]
23 May 15:48    INFO  epoch 78 evaluating [time: 

Train    88:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  62%|██████████████████████████████▋                  | 25/40 [00:00<00:00, 247.94it/s]: 100%|█████████████████████████████████████████████████| 40/40 [00:00<00:00, 224.43it/s]
23 May 15:48    INFO  epoch 88 training [time: 0.19s, train loss: 0.0002]
Evaluate   :   0%|                                                            | 0/3 [00:00<?, ?it/s]: 100%|███████████████████████████████████████████████████| 3/3 [00:00<00:00, 183.11it/s]
23 May 15:48    INFO  epoch 88 evaluating [time: 0.03s, valid_score: 0.008200]
23 May 15:48    INFO  valid result: 
rmse : 0.0082    mae : 0.0003
23 May 15:48    INFO  Saving current: saved/FM-May-23-2024_15-48-38.pth
Train    89:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  60%|█████████████████████████████▍                   | 24/40 [00:00<00:00, 233.21it/s]: 100%|███████████████████████████████████████████

Evaluate   :   0%|                                                            | 0/3 [00:00<?, ?it/s]: 100%|███████████████████████████████████████████████████| 3/3 [00:00<00:00, 169.95it/s]
23 May 15:49    INFO  epoch 98 evaluating [time: 0.03s, valid_score: 0.008200]
23 May 15:49    INFO  valid result: 
rmse : 0.0082    mae : 0.0003
23 May 15:49    INFO  Saving current: saved/FM-May-23-2024_15-48-38.pth
Train    99:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  57%|████████████████████████████▏                    | 23/40 [00:00<00:00, 227.12it/s]: 100%|█████████████████████████████████████████████████| 40/40 [00:00<00:00, 242.90it/s]
23 May 15:49    INFO  epoch 99 training [time: 0.17s, train loss: 0.0002]
Evaluate   :   0%|                                                            | 0/3 [00:00<?, ?it/s]: 100%|███████████████████████████████████████████████████| 3/3 [00:00<00:00, 363.19it/s]
23 May 15:49    INFO  epoch 99 evaluating [time: 

{'best_valid_score': 0.0082,
 'valid_score_bigger': False,
 'best_valid_result': OrderedDict([('rmse', 0.0082), ('mae', 0.0003)]),
 'test_result': OrderedDict([('rmse', 0.0105), ('mae', 0.0005)])}

3. **AFM**:
The Attentional Factorization Machine (AFM) model was implemented in the RecBole framework to evaluate its performance with both explicit and implicit feedback. For explicit feedback, ratings were normalized between 0 and 1, with the model evaluated using RMSE and MAE metrics, achieving an RMSE of 0.1843 and MAE of 0.1449 on the test set. Conversely, for implicit feedback, ratings were converted to binary indicators of interaction, and the model similarly utilized RMSE and MAE for evaluation, resulting in an RMSE of 0.0119 and MAE of 0.0006.

AFM incorporates attention mechanisms to prioritize significant feature interactions, providing nuanced insights into user-item relationships and demonstrating versatility in handling various feedback types by adjusting preprocessing and evaluation metrics. However, the model was not assessed using top-k ranking metrics in the explicit feedback setup, which might limit its applicability for tasks focused on ranking recommendations. Additionally, the performance sensitivity to different types of feedback suggests that further tuning might be required for optimal application in real-world scenarios, particularly when transitioning between explicit and implicit feedback settings.

In [5]:
import pandas as pd
from recbole.quick_start import run_recbole

# Load the data
data = pd.read_csv('u.data', sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])

# Normalize ratings between 0 and 1
max_rating = data['rating'].max()
data['rating'] = data['rating'] / max_rating

# Save processed data for RecBole
data.to_csv('dataset/ml100k_explicit_afm/ml100k_explicit_afm.inter', index=False, sep='\t',
            header=['user_id:token', 'item_id:token', 'rating:float', 'timestamp:float'])

# Configuration for AFM with Explicit Feedback
config_dict_explicit = {
    'model': 'AFM',
    'dataset': 'ml100k_explicit_afm',
    'epochs': 100,
    'eval_setting': 'RO_RS',  # Random Ordering, Random Splitting
    'metrics': ['RMSE', 'MAE'],  # Evaluation metrics suitable for binary classification
    'valid_metric': 'RMSE',  # Main metric for validation
    'topk': 10,
    'early_stop': 5,
    'gpu_id': 0,
    'LABEL_FIELD': 'rating',
    'load_col': {'inter': ['user_id', 'item_id', 'rating', 'timestamp']},
    'neg_sampling': None,  # Make sure this is appropriate for your setup
    'ITEM_ID_FIELD': 'item_id'  # Ensuring item_id is recognized correctly
}


# Run the model
run_recbole(model='AFM', dataset='ml100k_explicit_afm', config_dict=config_dict_explicit)

23 May 15:51    INFO  ['/home/stef/.local/lib/python3.10/site-packages/ipykernel_launcher.py', '-f', '/home/stef/.local/share/jupyter/runtime/kernel-510548df-5f9c-4af3-8da9-9e6950dbae6c.json']
23 May 15:51    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/ml100k_explicit_afm
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 100
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'RO', 'group_by': None, 'mode': {'valid': 'labeled', 'test': 'labeled'}

23 May 15:51    INFO  epoch 3 training [time: 0.31s, train loss: 206.9634]
Evaluate   :   0%|                                                            | 0/3 [00:00<?, ?it/s]: 100%|███████████████████████████████████████████████████| 3/3 [00:00<00:00, 252.42it/s]
23 May 15:51    INFO  epoch 3 evaluating [time: 0.02s, valid_score: 0.247200]
23 May 15:51    INFO  valid result: 
rmse : 0.2472    mae : 0.2042
23 May 15:51    INFO  Saving current: saved/AFM-May-23-2024_15-51-17.pth
Train     4:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  50%|████████████████████████▌                        | 20/40 [00:00<00:00, 195.74it/s]: 100%|█████████████████████████████████████████████████| 40/40 [00:00<00:00, 179.93it/s]: 100%|█████████████████████████████████████████████████| 40/40 [00:00<00:00, 178.79it/s]
23 May 15:51    INFO  epoch 4 training [time: 0.23s, train loss: 177.6313]
Evaluate   :   0%|                                                        

Train    13:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  45%|██████████████████████                           | 18/40 [00:00<00:00, 176.90it/s]:  98%|███████████████████████████████████████████████▊ | 39/40 [00:00<00:00, 193.74it/s]: 100%|█████████████████████████████████████████████████| 40/40 [00:00<00:00, 185.35it/s]
23 May 15:51    INFO  epoch 13 training [time: 0.23s, train loss: 23.0058]
Evaluate   :   0%|                                                            | 0/3 [00:00<?, ?it/s]: 100%|███████████████████████████████████████████████████| 3/3 [00:00<00:00, 189.80it/s]
23 May 15:51    INFO  epoch 13 evaluating [time: 0.03s, valid_score: 0.191700]
23 May 15:51    INFO  valid result: 
rmse : 0.1917    mae : 0.1528
23 May 15:51    INFO  Saving current: saved/AFM-May-23-2024_15-51-17.pth
Train    14:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  42%|████████████████████▊                    

23 May 15:51    INFO  epoch 22 evaluating [time: 0.04s, valid_score: 0.186300]
23 May 15:51    INFO  valid result: 
rmse : 0.1863    mae : 0.1477
23 May 15:51    INFO  Saving current: saved/AFM-May-23-2024_15-51-17.pth
Train    23:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  48%|███████████████████████▎                         | 19/40 [00:00<00:00, 185.01it/s]:  95%|██████████████████████████████████████████████▌  | 38/40 [00:00<00:00, 180.54it/s]: 100%|█████████████████████████████████████████████████| 40/40 [00:00<00:00, 173.21it/s]
23 May 15:51    INFO  epoch 23 training [time: 0.24s, train loss: 22.3870]
Evaluate   :   0%|                                                            | 0/3 [00:00<?, ?it/s]: 100%|███████████████████████████████████████████████████| 3/3 [00:00<00:00, 178.55it/s]
23 May 15:51    INFO  epoch 23 evaluating [time: 0.03s, valid_score: 0.185900]
23 May 15:51    INFO  valid result: 
rmse : 0.1859    mae : 0.1473
23

Evaluate   :   0%|                                                            | 0/3 [00:00<?, ?it/s]: 100%|███████████████████████████████████████████████████| 3/3 [00:00<00:00, 268.17it/s]
23 May 15:51    INFO  epoch 32 evaluating [time: 0.02s, valid_score: 0.184000]
23 May 15:51    INFO  valid result: 
rmse : 0.184    mae : 0.1453
23 May 15:51    INFO  Saving current: saved/AFM-May-23-2024_15-51-17.pth
Train    33:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  50%|████████████████████████▌                        | 20/40 [00:00<00:00, 190.48it/s]: 100%|█████████████████████████████████████████████████| 40/40 [00:00<00:00, 151.85it/s]: 100%|█████████████████████████████████████████████████| 40/40 [00:00<00:00, 153.62it/s]
23 May 15:51    INFO  epoch 33 training [time: 0.27s, train loss: 22.2166]
Evaluate   :   0%|                                                            | 0/3 [00:00<?, ?it/s]: 100%|██████████████████████████████████████████

Train    42:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  48%|███████████████████████▎                         | 19/40 [00:00<00:00, 184.72it/s]:  95%|██████████████████████████████████████████████▌  | 38/40 [00:00<00:00, 169.63it/s]: 100%|█████████████████████████████████████████████████| 40/40 [00:00<00:00, 166.80it/s]
23 May 15:51    INFO  epoch 42 training [time: 0.25s, train loss: 22.0817]
Evaluate   :   0%|                                                            | 0/3 [00:00<?, ?it/s]: 100%|███████████████████████████████████████████████████| 3/3 [00:00<00:00, 243.30it/s]
23 May 15:51    INFO  epoch 42 evaluating [time: 0.02s, valid_score: 0.183400]
23 May 15:51    INFO  valid result: 
rmse : 0.1834    mae : 0.1443
23 May 15:51    INFO  Saving current: saved/AFM-May-23-2024_15-51-17.pth
Train    43:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  45%|██████████████████████                   

Evaluate   :   0%|                                                            | 0/3 [00:00<?, ?it/s]: 100%|███████████████████████████████████████████████████| 3/3 [00:00<00:00, 180.85it/s]
23 May 15:51    INFO  epoch 51 evaluating [time: 0.04s, valid_score: 0.183400]
23 May 15:51    INFO  valid result: 
rmse : 0.1834    mae : 0.1441
Train    52:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  42%|████████████████████▊                            | 17/40 [00:00<00:00, 163.97it/s]:  85%|█████████████████████████████████████████▋       | 34/40 [00:00<00:00, 155.79it/s]: 100%|█████████████████████████████████████████████████| 40/40 [00:00<00:00, 154.69it/s]
23 May 15:51    INFO  epoch 52 training [time: 0.27s, train loss: 21.9620]
Evaluate   :   0%|                                                            | 0/3 [00:00<?, ?it/s]: 100%|███████████████████████████████████████████████████| 3/3 [00:00<00:00, 244.32it/s]
23 May 15:51    INFO  epoch 52 

23 May 15:51    INFO  Loading model structure and parameters from saved/AFM-May-23-2024_15-51-17.pth
Evaluate   :   0%|                                                            | 0/3 [00:00<?, ?it/s]: 100%|███████████████████████████████████████████████████| 3/3 [00:00<00:00, 299.28it/s]
23 May 15:51    INFO  The running environment of this training is as follows:
+-------------+----------------+
| Environment |     Usage      |
+=============+================+
| CPU         |    34.80 %     |
+-------------+----------------+
| GPU         |   0.0 / 0.0    |
+-------------+----------------+
| Memory      | 0.54 G/13.86 G |
+-------------+----------------+
23 May 15:51    INFO  best valid : OrderedDict([('rmse', 0.1833), ('mae', 0.144)])
23 May 15:51    INFO  test result: OrderedDict([('rmse', 0.1843), ('mae', 0.1449)])


{'best_valid_score': 0.1833,
 'valid_score_bigger': False,
 'best_valid_result': OrderedDict([('rmse', 0.1833), ('mae', 0.144)]),
 'test_result': OrderedDict([('rmse', 0.1843), ('mae', 0.1449)])}

In [6]:
import pandas as pd
from recbole.quick_start import run_recbole

# Load data and treat any rating as an interaction (implicit feedback)
data = pd.read_csv('u.data', sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])
data['rating'] = (data['rating'] > 0).astype(int)  # Convert ratings to 1 to denote interaction

# Save the data with simple headers
data.to_csv('dataset/ml100k_implicit_afm/ml100k_implicit_afm.inter', index=False, sep='\t',
            header=['user_id:token', 'item_id:token', 'rating:float', 'timestamp:float'])

# Configuration for AFM with implicit feedback
config_dict_implicit = {
    'model': 'AFM',
    'dataset': 'ml100k_implicit_afm',
    'epochs': 100,
    'learning_rate': 0.01,
    'eval_setting': 'RO_RS',  # Random Ordering, Random Splitting
    'metrics': ['RMSE', 'MAE'],  # Evaluation metrics suitable for binary classification
    'valid_metric': 'RMSE',  # Main metric for validation
    'topk': 10,
    'early_stop': 5,
    'gpu_id': 0,
    'LABEL_FIELD': 'rating',  # This must match exactly with your data headers
    'load_col': {'inter': ['user_id', 'item_id', 'rating', 'timestamp']}
}

# Run the model
run_recbole(model='AFM', dataset='ml100k_implicit_afm', config_dict=config_dict_implicit)

23 May 15:52    INFO  ['/home/stef/.local/lib/python3.10/site-packages/ipykernel_launcher.py', '-f', '/home/stef/.local/share/jupyter/runtime/kernel-510548df-5f9c-4af3-8da9-9e6950dbae6c.json']
23 May 15:52    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/ml100k_implicit_afm
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 100
train_batch_size = 2048
learner = adam
learning_rate = 0.01
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'RO', 'group_by': None, 'mode': {'valid': 'labeled', 'test': 'labeled'}}

Evaluate   :   0%|                                                            | 0/3 [00:00<?, ?it/s]: 100%|███████████████████████████████████████████████████| 3/3 [00:00<00:00, 219.86it/s]
23 May 15:52    INFO  epoch 3 evaluating [time: 0.03s, valid_score: 0.009900]
23 May 15:52    INFO  valid result: 
rmse : 0.0099    mae : 0.0014
23 May 15:52    INFO  Saving current: saved/AFM-May-23-2024_15-52-12.pth
Train     4:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  42%|████████████████████▊                            | 17/40 [00:00<00:00, 162.43it/s]:  85%|█████████████████████████████████████████▋       | 34/40 [00:00<00:00, 164.35it/s]: 100%|█████████████████████████████████████████████████| 40/40 [00:00<00:00, 165.63it/s]
23 May 15:52    INFO  epoch 4 training [time: 0.25s, train loss: 0.6689]
Evaluate   :   0%|                                                            | 0/3 [00:00<?, ?it/s]: 100%|████████████████████████████████████████████

Train    13:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  50%|████████████████████████▌                        | 20/40 [00:00<00:00, 191.89it/s]: 100%|█████████████████████████████████████████████████| 40/40 [00:00<00:00, 179.18it/s]: 100%|█████████████████████████████████████████████████| 40/40 [00:00<00:00, 176.55it/s]
23 May 15:52    INFO  epoch 13 training [time: 0.24s, train loss: 0.5484]
Evaluate   :   0%|                                                            | 0/3 [00:00<?, ?it/s]: 100%|███████████████████████████████████████████████████| 3/3 [00:00<00:00, 214.10it/s]
23 May 15:52    INFO  epoch 13 evaluating [time: 0.03s, valid_score: 0.009100]
23 May 15:52    INFO  valid result: 
rmse : 0.0091    mae : 0.0004
23 May 15:52    INFO  Saving current: saved/AFM-May-23-2024_15-52-12.pth
Train    14:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  45%|██████████████████████                    

Evaluate   :   0%|                                                            | 0/3 [00:00<?, ?it/s]: 100%|███████████████████████████████████████████████████| 3/3 [00:00<00:00, 248.26it/s]
23 May 15:52    INFO  epoch 22 evaluating [time: 0.03s, valid_score: 0.009000]
23 May 15:52    INFO  valid result: 
rmse : 0.009    mae : 0.0003
23 May 15:52    INFO  Saving current: saved/AFM-May-23-2024_15-52-12.pth
Train    23:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  45%|██████████████████████                           | 18/40 [00:00<00:00, 173.94it/s]:  90%|████████████████████████████████████████████     | 36/40 [00:00<00:00, 168.32it/s]: 100%|█████████████████████████████████████████████████| 40/40 [00:00<00:00, 166.96it/s]
23 May 15:52    INFO  epoch 23 training [time: 0.25s, train loss: 0.5822]
Evaluate   :   0%|                                                            | 0/3 [00:00<?, ?it/s]: 100%|███████████████████████████████████████████

Train    32:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  38%|██████████████████▍                              | 15/40 [00:00<00:00, 139.86it/s]:  75%|████████████████████████████████████▊            | 30/40 [00:00<00:00, 141.59it/s]: 100%|█████████████████████████████████████████████████| 40/40 [00:00<00:00, 144.62it/s]
23 May 15:52    INFO  epoch 32 training [time: 0.29s, train loss: 0.5761]
Evaluate   :   0%|                                                            | 0/3 [00:00<?, ?it/s]: 100%|███████████████████████████████████████████████████| 3/3 [00:00<00:00, 192.95it/s]
23 May 15:52    INFO  epoch 32 evaluating [time: 0.03s, valid_score: 0.008900]
23 May 15:52    INFO  valid result: 
rmse : 0.0089    mae : 0.0003
23 May 15:52    INFO  Saving current: saved/AFM-May-23-2024_15-52-12.pth
Train    33:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  40%|███████████████████▌                      

...

Evaluate   :   0%|                                                            | 0/3 [00:00<?, ?it/s]: 100%|███████████████████████████████████████████████████| 3/3 [00:00<00:00, 291.24it/s]
23 May 15:52    INFO  epoch 71 evaluating [time: 0.02s, valid_score: 0.008700]
23 May 15:52    INFO  valid result: 
rmse : 0.0087    mae : 0.0003
23 May 15:52    INFO  Saving current: saved/AFM-May-23-2024_15-52-12.pth
Train    72:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  52%|█████████████████████████▋                       | 21/40 [00:00<00:00, 207.17it/s]: 100%|█████████████████████████████████████████████████| 40/40 [00:00<00:00, 199.78it/s]
23 May 15:52    INFO  epoch 72 training [time: 0.21s, train loss: 0.5883]
Evaluate   :   0%|                                                            | 0/3 [00:00<?, ?it/s]: 100%|███████████████████████████████████████████████████| 3/3 [00:00<00:00, 241.43it/s]
23 May 15:52    INFO  epoch 72 evaluating [time:

23 May 15:52    INFO  epoch 81 training [time: 0.31s, train loss: 0.5797]
Evaluate   :   0%|                                                            | 0/3 [00:00<?, ?it/s]: 100%|███████████████████████████████████████████████████| 3/3 [00:00<00:00, 209.41it/s]
23 May 15:52    INFO  epoch 81 evaluating [time: 0.04s, valid_score: 0.008700]
23 May 15:52    INFO  valid result: 
rmse : 0.0087    mae : 0.0003
23 May 15:52    INFO  Saving current: saved/AFM-May-23-2024_15-52-12.pth
Train    82:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  18%|████████▉                                          | 7/40 [00:00<00:00, 66.96it/s]:  45%|██████████████████████▌                           | 18/40 [00:00<00:00, 78.08it/s]:  72%|████████████████████████████████████▎             | 29/40 [00:00<00:00, 90.69it/s]: 100%|██████████████████████████████████████████████████| 40/40 [00:00<00:00, 98.38it/s]
23 May 15:52    INFO  epoch 82 training [time: 0.42s, train 

Train    91:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  42%|████████████████████▊                            | 17/40 [00:00<00:00, 168.15it/s]:  88%|██████████████████████████████████████████▉      | 35/40 [00:00<00:00, 168.63it/s]: 100%|█████████████████████████████████████████████████| 40/40 [00:00<00:00, 158.02it/s]
23 May 15:52    INFO  epoch 91 training [time: 0.27s, train loss: 0.6026]
Evaluate   :   0%|                                                            | 0/3 [00:00<?, ?it/s]: 100%|████████████████████████████████████████████████████| 3/3 [00:00<00:00, 92.40it/s]
23 May 15:52    INFO  epoch 91 evaluating [time: 0.05s, valid_score: 0.008600]
23 May 15:52    INFO  valid result: 
rmse : 0.0086    mae : 0.0003
23 May 15:52    INFO  Saving current: saved/AFM-May-23-2024_15-52-12.pth
Train    92:   0%|                                                           | 0/40 [00:00<?, ?it/s]:  38%|██████████████████▍                       

+-------------+----------------+
23 May 15:52    INFO  best valid : OrderedDict([('rmse', 0.0086), ('mae', 0.0003)])
23 May 15:52    INFO  test result: OrderedDict([('rmse', 0.0119), ('mae', 0.0006)])


{'best_valid_score': 0.0086,
 'valid_score_bigger': False,
 'best_valid_result': OrderedDict([('rmse', 0.0086), ('mae', 0.0003)]),
 'test_result': OrderedDict([('rmse', 0.0119), ('mae', 0.0006)])}